### 1. 学会看
先让AI读懂藏文吧！看acc一步一步往上爬

In [1]:
from tensorflow import keras
from keras import utils
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


#### 1.1 全连接神经网络
MLP test acc: 97.44%

In [18]:
#数据加载
data = np.load('/home/kesci/input/TibetanMNIST5610/TibetanMNIST.npz')
X, y = data['image'], data['label']  # (17768, 28, 28)
X = X.reshape(17768, 784).astype('float32') / 255
y = np_utils.to_categorical(y, num_classes=10)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state=2018)

#MLP
model = Sequential()
model.add(Dense(784, input_dim=784, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam' , loss = "categorical_crossentropy", metrics=["accuracy"])
model.summary()

history=model.fit(x=X_train, y=y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))
evaluation = model.evaluate(X_test, y_test, verbose=1)
print('Summary: Loss over the test dataset: %.4f, Accuracy: %.4f' % (evaluation[0], evaluation[1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 784)               615440    
_________________________________________________________________
dense_34 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 10)                2570      
Total params: 1,151,258
Trainable params: 1,151,258
Non-trainable params: 0
_________________________________________________________________


#### 1.2 卷积神经网络 
CNN test acc: 99.16%

In [19]:
#CNN
batch_size = 128
epochs = 10
data = np.load('/home/kesci/input/TibetanMNIST5610/TibetanMNIST.npz')
X, y = data['image'], data['label']  # (17768, 28, 28)
X = X.reshape(X.shape[0], 28,28,1).astype('float32') / 255
X_train, X_test, y_train, y_test_ = train_test_split(X, y, test_size=0.2, random_state=2018)
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test_, num_classes=10)

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.compile(optimizer='adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, 
          validation_data = (X_test, y_test), verbose = 2)
          
evaluation = model.evaluate(X_test, y_test, verbose=1)
print('Summary: Loss over the test dataset: %.4f, Accuracy: %.4f' % (evaluation[0], evaluation[1]))

Train on 14214 samples, validate on 3554 samples
Epoch 1/10
 - 252s - loss: 0.5506 - acc: 0.8146 - val_loss: 0.1384 - val_acc: 0.9524
Epoch 2/10
 - 277s - loss: 0.1609 - acc: 0.9466 - val_loss: 0.0729 - val_acc: 0.9761
Epoch 3/10
 - 278s - loss: 0.1109 - acc: 0.9623 - val_loss: 0.0529 - val_acc: 0.9840
Epoch 4/10
 - 240s - loss: 0.0874 - acc: 0.9720 - val_loss: 0.0478 - val_acc: 0.9851
Epoch 5/10
 - 204s - loss: 0.0660 - acc: 0.9783 - val_loss: 0.0407 - val_acc: 0.9871
Epoch 6/10
 - 214s - loss: 0.0531 - acc: 0.9842 - val_loss: 0.0342 - val_acc: 0.9887
Epoch 7/10
 - 276s - loss: 0.0513 - acc: 0.9837 - val_loss: 0.0371 - val_acc: 0.9893
Epoch 8/10
 - 276s - loss: 0.0459 - acc: 0.9860 - val_loss: 0.0411 - val_acc: 0.9868
Epoch 9/10
 - 269s - loss: 0.0417 - acc: 0.9870 - val_loss: 0.0323 - val_acc: 0.9896
Epoch 10/10
 - 201s - loss: 0.0341 - acc: 0.9892 - val_loss: 0.0326 - val_acc: 0.9916
3554/3554 [==============================] - 32s 9ms/step
Summary: Loss over the test dataset: 0.032

#### 1.3 CNN模型调优 
CNN test acc: 99.24%
 - 数据增强
 - 添加一个学习率衰减器
 
> 加大epoch数值，精度仍有上升空间，好慢呀

In [20]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
datagen = ImageDataGenerator()
datagen.fit(X_train)
# Fit the model
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test,y_test),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size,
                              callbacks=[learning_rate_reduction])
evaluation = model.evaluate(X_test, y_test, verbose=1)
print('Summary: Loss over the test dataset: %.4f, Accuracy: %.4f' % (evaluation[0], evaluation[1]))

Epoch 1/10
 - 206s - loss: 0.0290 - acc: 0.9905 - val_loss: 0.0372 - val_acc: 0.9893
Epoch 2/10
 - 197s - loss: 0.0267 - acc: 0.9913 - val_loss: 0.0365 - val_acc: 0.9893
Epoch 3/10
 - 230s - loss: 0.0271 - acc: 0.9910 - val_loss: 0.0326 - val_acc: 0.9893
Epoch 4/10
 - 276s - loss: 0.0253 - acc: 0.9918 - val_loss: 0.0365 - val_acc: 0.9904
Epoch 5/10
 - 277s - loss: 0.0264 - acc: 0.9907 - val_loss: 0.0299 - val_acc: 0.9921
Epoch 6/10
 - 244s - loss: 0.0213 - acc: 0.9935 - val_loss: 0.0291 - val_acc: 0.9916
Epoch 7/10
 - 197s - loss: 0.0238 - acc: 0.9925 - val_loss: 0.0305 - val_acc: 0.9916
Epoch 8/10
 - 212s - loss: 0.0186 - acc: 0.9937 - val_loss: 0.0360 - val_acc: 0.9904
Epoch 9/10
 - 276s - loss: 0.0249 - acc: 0.9914 - val_loss: 0.0296 - val_acc: 0.9916

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 10/10
 - 277s - loss: 0.0129 - acc: 0.9961 - val_loss: 0.0281 - val_acc: 0.9924
3554/3554 [==============================] - 9s 3ms/step
Summary: Lo

### 2. 学会写
识别是学习的第一步，下面我们GAN起来，看看AI书写的藏文数字长啥样！

自2014年Ian Goodfellow提出生成对抗网络(GAN)的概念后,生成对抗网络变成为了学术界的一个火热的研究热点,Yann LeCun更是称之为”过去十年间机器学习领域最让人激动的点子”.生成对抗网络的简单思路如下,训练一个生成器(Generator,简称G),从随机噪声或者潜在变量(Latent Variable)中生成逼真的的样本,同时训练一个鉴别器(Discriminator,简称D)来鉴别真实数据和生成数据,两者同时训练,直到达到一个纳什均衡,生成器生成的数据与真实样本无差别,鉴别器也无法正确的区分生成数据和真实数据.GAN的结构如图1所示.

![Image Name](https://cdn.kesci.com/upload/image/pj9uwejw6c.png?imageView2/0/w/960/h/960)


github有一个库使用Keras对21种GAN及其变体作了精简的实现，我们略作修改，看看效果如何
> https://github.com/eriklindernoren/Keras-GAN

#### 2.1 CGAN 

In [14]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        #(X_train, y_train), (_, _) = mnist.load_data()
        data = np.load('/home/kesci/input/TibetanMNIST5610/TibetanMNIST.npz')

        # Configure input
        #X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        #X_train = np.expand_dims(X_train, axis=3)
        #y_train = y_train.reshape(-1, 1)
        X, y = data['image'], data['label']  # (17768, 28, 28)
        X_train = X.reshape(X.shape[0], 28,28,1).astype('float32') / 255
        y_train = y.reshape(-1, 1)
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)
            if epoch % 200 ==0:
                # Plot the progress
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch, is_save=False):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        if is_save == True:
            fig.savefig("images2/%d.png" % epoch)
        else:
            plt.show()
        plt.close()



In [15]:
batch_size = 128
epochs = 20000
cgan = CGAN()
cgan.train(epochs=epochs, batch_size=batch_size, sample_interval=1000)#sample_interval 1000epoch输出一次图像


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 512)               0         
__________

/opt/conda/lib/python3.5/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.693387, acc.: 37.89%] [G loss: 0.676710]


<Figure size 432x288 with 10 Axes>

200 [D loss: 0.515204, acc.: 77.73%] [G loss: 1.695024]
400 [D loss: 0.671725, acc.: 58.20%] [G loss: 1.061109]
600 [D loss: 0.604611, acc.: 66.41%] [G loss: 1.114438]
800 [D loss: 0.509307, acc.: 78.12%] [G loss: 1.131666]
1000 [D loss: 0.599375, acc.: 66.02%] [G loss: 1.139028]


...

17200 [D loss: 0.009672, acc.: 99.61%] [G loss: 3.650989]
17400 [D loss: 0.032112, acc.: 98.83%] [G loss: 1.075614]
17600 [D loss: 0.006818, acc.: 100.00%] [G loss: 3.086496]
17800 [D loss: 0.002379, acc.: 100.00%] [G loss: 7.016960]
18000 [D loss: 0.017629, acc.: 99.61%] [G loss: 1.718827]


<Figure size 432x288 with 10 Axes>

18200 [D loss: 0.001037, acc.: 100.00%] [G loss: 0.515290]
18400 [D loss: 0.013190, acc.: 99.61%] [G loss: 5.891022]
18600 [D loss: 0.028058, acc.: 98.83%] [G loss: 8.746876]
18800 [D loss: 0.017850, acc.: 99.61%] [G loss: 6.442552]
19000 [D loss: 0.015755, acc.: 99.22%] [G loss: 3.640223]


<Figure size 432x288 with 10 Axes>

19200 [D loss: 0.017193, acc.: 98.83%] [G loss: 8.695421]
19400 [D loss: 0.020689, acc.: 99.22%] [G loss: 11.479371]
19600 [D loss: 0.041356, acc.: 98.83%] [G loss: 4.472672]
19800 [D loss: 0.030090, acc.: 98.44%] [G loss: 2.904464]


可以看见效果不算很好，由于计算效率较低，不再加大epoch实验了

#### 2.2 其他GAN 
我们可以在上面的库中使用其他GAN进行实验，WGAN效果会比CGAN好一些


### 3 搞点创作

#### 3.1 变成彩色
手动把灰度图变成彩色图

In [36]:
# 随机抽取风格图片的部分像素作为原图的背景
def coloful_batch(X_train,y_train, batch_size, style,change_colors=False):
    # Select random batch (WxHxC)
    idx = np.random.choice(X_train.shape[0], batch_size)
    labels = y_train[idx]
    batch_raw = X_train[idx, :, :, 0].reshape((batch_size, 28, 28, 1))
    
    # Resize
    batch_resized = np.asarray([scipy.ndimage.zoom(image, (2.3, 2.3, 1), order=1) for image in batch_raw])
    
    # Extend to RGB
    batch_rgb = np.concatenate([batch_resized, batch_resized, batch_resized], axis=3)
    
    # Make binary
    batch_binary = (batch_rgb > 0.5)
    
    batch = np.zeros((batch_size, 64, 64, 3))
    
    for i in range(batch_size):
        # Take a random crop of the style image (background)
        x_c = np.random.randint(0, style.size[0] - 64)
        y_c = np.random.randint(0, style.size[1] - 64)
        image = style.crop((x_c, y_c, x_c + 64, y_c + 64))
        image = np.asarray(image) / 255.0

        if change_colors:
            # Change color distribution
            for j in range(3):
                image[:, :, j] = (image[:, :, j] + np.random.uniform(0, 1)) / 2.0

        # Invert the colors at the location of the number
        image[:, :, 0:3][batch_binary[i]] = 1 - image[:, :, 0:3][batch_binary[i]]
        #image[batch_binary[i]] = 1 - image[batch_binary[i]]
        
        batch[i] = image[:, :, 0:3]
    return batch,labels

In [29]:
import scipy.ndimage
import numpy as np
import matplotlib.pyplot as plt
import PIL
style_img = PIL.Image.open('/home/kesci/work/style.jpg') #使用我的桌面壁纸来做风格图片吧~
plt.imshow(style)
plt.axis('off')
plt.show()


<Figure size 432x288 with 1 Axes>

In [49]:
data = np.load('/home/kesci/input/TibetanMNIST5610/TibetanMNIST.npz')
X, y = data['image'], data['label']  
X_train = X.reshape(X.shape[0], 28,28,1).astype('float32') / 255
y_train = y.reshape(-1, 1)
#随机查看20张生成的图片
imgs,labels = coloful_batch(X_train,y_train, batch_size = 20, style = style_img)
fig = plt.figure(figsize=(15,3))
for i in range(20):
    ax = fig.add_subplot(2, 20 // 2, i+1)
    ax.imshow(imgs[i])
    ax.set_title("%d" % labels[i])
    ax.axis('off')
    
plt.tight_layout()
plt.show()


<Figure size 1080x216 with 20 Axes>

用这种方法得到彩图之后，我们还可以做一些有趣的事情
- 我们可以再训练一次GAN，这样比起直接拿灰度图去生成效果会好看点。
- 我们可以再对其进行图像识别，由于揉入了风格图片的像素，相当于提高了识别的难度。

#### 3.2 风格迁移
我们来进行真正的风格迁移吧
> 待续